In [16]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.datasets import load_digits, make_blobs
from matplotlib.dates import MonthLocator, DateFormatter
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import AgglomerativeClustering

from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict
# To tune a model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from utils import (
    col_out,
    model_summary,
    adj_r2_score,
    model_performance_regression,
    confusion_matrix_helper,
    model_performance_classification_sklearn,
)

import sklearn.metrics as metrics
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    roc_auc_score,
    precision_recall_curve,
    roc_curve,
    silhouette_score
)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import set_config

import xgboost as xgb

set_config(display='diagram')
# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)


In [17]:
#Dataset Analysis for pre-processing
bigfive_data = pd.read_csv('big_five.csv', delimiter = '\t')
bigfive_df = pd.DataFrame(data=bigfive_data)


In [18]:
bigfive_df.describe().T

,count,mean,std,min,25%,50%,75%,max
EXT1,1013558.00,2.65,1.26,0.00,1.00,3.00,4.00,5.00
EXT2,1013558.00,2.77,1.32,0.00,2.00,3.00,4.00,5.00
EXT3,1013558.00,3.29,1.22,0.00,2.00,3.00,4.00,5.00
EXT4,1013558.00,3.14,1.24,0.00,2.00,3.00,4.00,5.00
EXT5,1013558.00,3.28,1.28,0.00,2.00,3.00,4.00,5.00
EXT6,1013558.00,2.40,1.23,0.00,1.00,2.00,3.00,5.00
EXT7,1013558.00,2.77,1.40,0.00,2.00,3.00,4.00,5.00
EXT8,1013558.00,3.41,1.27,0.00,2.00,4.00,4.00,5.00
EXT9,1013558.00,2.96,1.35,0.00,2.00,3.00,4.00,5.00
EXT10,1013558.00,3.56,1.31,0.00,3.00,4.00,5.00,5.00


In [19]:
bigfive_df.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,EXT1_E,EXT2_E,EXT3_E,EXT4_E,EXT5_E,EXT6_E,EXT7_E,EXT8_E,EXT9_E,EXT10_E,EST1_E,EST2_E,EST3_E,EST4_E,EST5_E,EST6_E,EST7_E,EST8_E,EST9_E,EST10_E,AGR1_E,AGR2_E,AGR3_E,AGR4_E,AGR5_E,AGR6_E,AGR7_E,AGR8_E,AGR9_E,AGR10_E,CSN1_E,CSN2_E,CSN3_E,CSN4_E,CSN5_E,CSN6_E,CSN7_E,CSN8_E,CSN9_E,CSN10_E,OPN1_E,OPN2_E,OPN3_E,OPN4_E,OPN5_E,OPN6_E,OPN7_E,OPN8_E,OPN9_E,OPN10_E,dateload,screenw,screenh,introelapse,testelapse,endelapse,IPC,country,lat_appx_lots_of_err,long_appx_lots_of_err
0,4.00,1.00,5.00,2.00,5.00,1.00,5.00,2.00,4.00,1.00,1.00,4.00,4.00,2.00,2.00,2.00,2.00,2.00,3.00,2.00,2.00,5.00,2.00,4.00,2.00,3.00,2.00,4.00,3.00,4.00,3.00,4.00,3.00,2.00,2.00,4.00,4.00,2.00,4.00,4.00,5.00,1.00,4.00,1.00,4.00,1.00,5.00,3.00,4.00,5.00,9419.00,5491.00,3959.00,4821.00,5611.00,2756.00,2388.00,2113.00,5900.00,4110.00,6135.00,4150.00,5739.00,6364.00,3663.00,5070.00,5709.00,4285.00,2587.00,3997.00,4750.00,5475.00,11641.00,3115.00,3207.00,3260.00,10235.00,5897.00,1758.00,3081.00,6602.00,5457.00,1569.00,2129.00,3762.00,4420.00,9382.00,5286.00,4983.00,6339.00,3146.00,4067.00,2959.00,3411.00,2170.00,4920.00,4436.00,3116.00,2992.00,4354.00,2016-03-03 02:01:01,768.00,1024.00,9.00,234.00,6,1,GB,51.5448,0.1991
1,3.00,5.00,3.00,4.00,3.00,3.00,2.00,5.00,1.00,5.00,2.00,3.00,4.00,1.00,3.00,1.00,2.00,1.00,3.00,1.00,1.00,4.00,1.00,5.00,1.00,5.00,3.00,4.00,5.00,3.00,3.00,2.00,5.00,3.00,3.00,1.00,3.00,3.00,5.00,3.00,1.00,2.00,4.00,2.00,3.00,1.00,4.00,2.00,5.00,3.00,7235.00,3598.00,3315.00,2564.00,2976.00,3050.00,4787.00,3228.00,3465.00,3309.00,9036.00,2406.00,3484.00,3359.00,3061.00,2539.00,4226.00,2962.00,1799.00,1607.00,2158.00,2090.00,2143.00,2807.00,3422.00,5324.00,4494.00,3627.00,1850.00,1747.00,5163.00,5240.00,7208.00,2783.00,4103.00,3431.00,3347.00,2399.00,3360.00,5595.00,2624.00,4985.00,1684.00,3026.00,4742.00,3336.00,2718.00,3374.00,3096.00,3019.00,2016-03-03 02:01:20,1360.00,768.00,12.00,179.00,11,1,MY,3.1698,101.706
2,2.00,3.00,4.00,4.00,3.00,2.00,1.00,3.00,2.00,5.00,4.00,4.00,4.00,2.00,2.00,2.00,2.00,2.00,1.00,3.00,1.00,4.00,1.00,4.00,2.00,4.00,1.00,4.00,4.00,3.00,4.00,2.00,2.00,2.00,3.00,3.00,4.00,2.00,4.00,2.00,5.00,1.00,2.00,1.00,4.00,2.00,5.00,3.00,4.00,4.00,4657.00,3549.00,2543.00,3335.00,5847.00,2540.00,4922.00,3142.00,14621.00,2191.00,5128.00,3675.00,3442.00,4546.00,8275.00,2185.00,2164.00,1175.00,3813.00,1593.00,1089.00,2203.00,3386.00,1464.00,2562.00,1493.00,3067.00,13719.00,3892.00,4100.00,4286.00,4775.00,2713.00,2813.00,4237.00,6308.00,2690.00,1516.00,2379.00,2983.00,1930.00,1470.00,1644.00,1683.00,2229.00,8114.00,2043.00,6295.00,1585.00,2529.00,2016-03-03 02:01:56,1366.00,768.00,3.00,186.00,7,1,GB,54.9119,-1.3833
3,2.00,2.00,2.00,3.00,4.00,2.00,2.00,4.00,1.00,4.00,3.00,3.00,3.00,2.00,3.00,2.00,2.00,2.00,4.00,3.00,2.00,4.00,3.00,4.00,2.00,4.00,2.00,4.00,3.00,4.00,2.00,4.00,4.00,4.00,1.00,2.00,2.00,3.00,1.00,4.00,4.00,2.00,5.00,2.00,3.00,1.00,4.00,4.00,3.00,3.00,3996.00,2896.00,5096.00,4240.00,5168.00,5456.00,4360.00,4496.00,5240.00,4000.00,3736.00,4616.00,3015.00,2711.00,3960.00,4064.00,4208.00,2936.00,7336.00,3896.00,6062.00,11952.00,1040.00,2264.00,3664.00,3049.00,4912.00,7545.00,4632.00,6896.00,2824.00,520.00,2368.00,3225.00,2848.00,6264.00,3760.00,10472.00,3192.00,7704.00,3456.00,6665.00,1977.00,3728.00,4128.00,3776.00,2984.00,4192.00,3480.00,3257.00,2016-03-03 02:02:02,1920.00,1200.00,186.00,219.00,7,1,GB,51.75,-1.25
4,3.00,3.00,3.00,3.00,5.00,3.00,3.00,5.00,3.00,4.00,1.00,5.00,5.00,3.00,1.00,1.00,1.00,1.00,3.00,2.00,1.00,5.00,1.00,5.00,1.00,3.00,1.00,5.00,5.00,3.00,5.00,1.00,5.00,1.00,3.00,1.00,5.00,1.00,5.00,5.00,5.00,1.00,5.00,1.00,5.00,1.00,5.00,3.00,5.00,5.00,6004.00,3965.00,2721.00,3706.00,2968.00,2426.00,7339.00,3302.00,16819.00,3731.00,4740.00,2856.00,7461.00,2179.00,3324.00,2255.00,430

In [20]:
pd.set_option('display.max_rows', None)
print('How many participants from each countries participated in the survey? \r', bigfive_df.groupby(['country'])['country'].value_counts().sort_values(ascending=False))

 country
US      546403
GB       66596
CA       61849
AU       50030
PH       19847
IN       17491
DE       14095
NONE     13728
NZ       12992
NO       11417
MY       11355
MX       11152
SE       10493
NL        9785
SG        7686
ID        6489
BR        6245
FR        6145
DK        5512
IE        5409
IT        5319
ES        5008
PL        4659
FI        4340
RO        3858
BE        3824
ZA        3751
CO        3619
HK        3595
PK        3511
RU        3323
AR        3154
CH        3124
AE        3061
TR        2891
PT        2519
GR        2513
VN        2337
HR        2245
AT        2215
CL        2193
RS        2065
CZ        2014
TH        1971
JP        1933
PE        1659
KR        1593
HU        1506
IL        1432
KE        1405
CN        1340
BG        1271
VE        1260
EC        1146
LT        1101
SA        1097
EG        1033
EE        1020
SK         992
NG         952
TW         921
SI         866
LB         837
UA         747
LK         701
CR         650
N

In [21]:
# Data Preprocessing - Cleaning Dataset

print('Are there any missing values? ', bigfive_df.isnull().values.any())
print('How many missing values are there in the dataset? ', bigfive_df.isnull().values.sum())
bigfive_df.dropna(inplace=True)
print('Number of participants after eliminating missing values: ', len(bigfive_df))

bigfive_df = bigfive_df.drop_duplicates()
print('Number of participants after dropping duplicate values: ', len(bigfive_df))

# Removing rows with IPC (IP Count) greater than 1, as suggested by dataset author:
# For max cleanliness, only use records where this value is 1. High values can be because of shared networks (e.g. entire universities) or multiple submissions
bigfive_df = bigfive_df[bigfive_df['IPC'] <= 1]
print('Number of participants after eliminating IP count of more than 1: ', len(bigfive_df))

# Looking at the output of describe there are questions that have 0 as an answer, these are not valid and should be removed.
answer_columns = ['EXT1','EXT2','EXT3','EXT4','EXT5','EXT6','EXT7','EXT8','EXT9','EXT10',
                'EST1','EST2','EST3','EST4','EST5','EST6','EST7','EST8','EST9','EST10',
                'AGR1','AGR2','AGR3','AGR4','AGR5','AGR6','AGR7','AGR8','AGR9','AGR10',
                'CSN1','CSN2','CSN3','CSN4','CSN5','CSN6','CSN7','CSN8','CSN9','CSN10',
                'OPN1','OPN2','OPN3','OPN4','OPN5','OPN6','OPN7','OPN8','OPN9','OPN10']
bigfive_df = bigfive_df[~(bigfive_df[answer_columns] == 0).any(axis=1)]
print('Number of participants after removing answers with 0: ', len(bigfive_df))

# Removing rows with country as NONE
bigfive_df = bigfive_df[bigfive_df['country'] != 'NONE']
print('Number of participants after eliminating blank Country: ', len(bigfive_df))

# We're using data from countries that have more than 1000 participants
bigfive_df = bigfive_df.groupby(['country']).filter(lambda x: len(x) > 1000)
print('Number of participants after eliminating countries with less than 1000 participants: ', len(bigfive_df))



Are there any missing values?  True
How many missing values are there in the dataset?  186358
Number of participants after eliminating missing values:  1012050
Number of participants after dropping duplicate values:  1012050
Number of participants after eliminating IP count of more than 1:  694886
Number of participants after removing answers with 0:  602642
Number of participants after eliminating blank Country:  594453
Number of participants after eliminating countries with less than 1000 participants:  570277


In [32]:
# Let's look at number of participants per country for the cleaned dataset
print('How many participants from each countries participated in the survey? \r')
print(bigfive_df.groupby(['country'])['country'].value_counts().sort_values(ascending=False))


How many participants from each countries participated in the survey? 
country
USA    300493
GBR     43411
CAN     38337
AUS     30302
DEU     10895
IND      9847
PHL      9470
MEX      8275
NOR      7137
NLD      6910
SWE      6526
MYS      6330
NZL      5752
IDN      4928
SGP      4794
BRA      4767
FRA      4497
ITA      3813
ESP      3719
POL      3621
IRL      3501
FIN      3410
DNK      3345
ROU      2908
COL      2721
RUS      2536
BEL      2474
ARG      2430
HKG      2421
ZAF      2399
TUR      2063
PAK      1995
PRT      1948
GRC      1931
CHE      1842
ARE      1722
AUT      1717
CHL      1707
HRV      1643
VNM      1471
SRB      1431
JPN      1417
CZE      1363
THA      1362
PER      1295
HUN      1169
KOR      1156
ISR      1076
Name: count, dtype: int64


In [23]:
# Dataset uses the ISO-3166 country code and needs to match up to the ISO-A3 format used by geopandas map.
mapping_dict = {
    'US':'USA','GB':'GBR','CA':'CAN','AU':'AUS','DE':'DEU','IN':'IND','PH':'PHL',
    'MX':'MEX','NO':'NOR','NL':'NLD','SE':'SWE','MY':'MYS','NZ':'NZL','ID':'IDN',
    'SG':'SGP','BR':'BRA','FR':'FRA','IT':'ITA','ES':'ESP','PL':'POL','IE':'IRL',
    'FI':'FIN','DK':'DNK','RO':'ROU','CO':'COL','RU':'RUS','BE':'BEL','AR':'ARG',
    'HK':'HKG','ZA':'ZAF','TR':'TUR','PK':'PAK','PT':'PRT','GR':'GRC','CH':'CHE',
    'AE':'ARE','AT':'AUT','CL':'CHL','HR':'HRV','VN':'VNM','RS':'SRB','JP':'JPN',
    'CZ':'CZE','TH':'THA','PE':'PER','HU':'HUN','KR':'KOR','IL':'ISR',
}

# Replace 'country' with the actual column name in your DataFrame
bigfive_df['country'] = bigfive_df['country'].map(mapping_dict)


In [24]:
# Removing Columns (for this exercise)
bigfive_df = bigfive_df.drop(bigfive_df.columns[50:100], axis=1)
bigfive_df = bigfive_df.drop(columns=['IPC'])
bigfive_df = bigfive_df.drop(columns=['lat_appx_lots_of_err','long_appx_lots_of_err'])



In [25]:
bigfive_df.reset_index(drop=True, inplace=True)
bigfive_df.head()

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,EST1,EST2,EST3,EST4,EST5,EST6,EST7,EST8,EST9,EST10,AGR1,AGR2,AGR3,AGR4,AGR5,AGR6,AGR7,AGR8,AGR9,AGR10,CSN1,CSN2,CSN3,CSN4,CSN5,CSN6,CSN7,CSN8,CSN9,CSN10,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10,dateload,screenw,screenh,introelapse,testelapse,endelapse,country
0,4.00,1.00,5.00,2.00,5.00,1.00,5.00,2.00,4.00,1.00,1.00,4.00,4.00,2.00,2.00,2.00,2.00,2.00,3.00,2.00,2.00,5.00,2.00,4.00,2.00,3.00,2.00,4.00,3.00,4.00,3.00,4.00,3.00,2.00,2.00,4.00,4.00,2.00,4.00,4.00,5.00,1.00,4.00,1.00,4.00,1.00,5.00,3.00,4.00,5.00,2016-03-03 02:01:01,768.00,1024.00,9.00,234.00,6,GBR
1,3.00,5.00,3.00,4.00,3.00,3.00,2.00,5.00,1.00,5.00,2.00,3.00,4.00,1.00,3.00,1.00,2.00,1.00,3.00,1.00,1.00,4.00,1.00,5.00,1.00,5.00,3.00,4.00,5.00,3.00,3.00,2.00,5.00,3.00,3.00,1.00,3.00,3.00,5.00,3.00,1.00,2.00,4.00,2.00,3.00,1.00,4.00,2.00,5.00,3.00,2016-03-03 02:01:20,1360.00,768.00,12.00,179.00,11,MYS
2,2.00,3.00,4.00,4.00,3.00,2.00,1.00,3.00,2.00,5.00,4.00,4.00,4.00,2.00,2.00,2.00,2.00,2.00,1.00,3.00,1.00,4.00,1.00,4.00,2.00,4.00,1.00,4.00,4.00,3.00,4.00,2.00,2.00,2.00,3.00,3.00,4.00,2.00,4.00,2.00,5.00,1.00,2.00,1.00,4.00,2.00,5.00,3.00,4.00,4.00,2016-03-03 02:01:56,1366.00,768.00,3.00,186.00,7,GBR
3,2.00,2.00,2.00,3.00,4.00,2.00,2.00,4.00,1.00,4.00,3.00,3.00,3.00,2.00,3.00,2.00,2.00,2.00,4.00,3.00,2.00,4.00,3.00,4.00,2.00,4.00,2.00,4.00,3.00,4.00,2.00,4.00,4.00,4.00,1.00,2.00,2.00,3.00,1.00,4.00,4.00,2.00,5.00,2.00,3.00,1.00,4.00,4.00,3.00,3.00,2016-03-03 02:02:02,1920.00,1200.00,186.00,219.00,7,GBR
4,3.00,3.00,4.00,2.00,4.00,2.00,2.00,3.00,3.00,4.00,3.00,4.00,3.00,2.00,2.00,1.00,2.00,1.00,2.00,2.00,2.00,3.00,1.00,4.00,2.00,3.00,2.00,3.00,4.00,4.00,3.00,2.00,4.00,1.00,3.00,2.00,4.00,3.00,4.00,3.00,5.00,1.00,5.00,1.00,3.00,1.00,5.00,4.00,5.00,2.00,2016-03-03 02:03:12,1600.00,1000.00,4.00,196.00,3,SWE


In [26]:
%%time
#Lets find out scores for each personality trait
#Based on https://openpsychometrics.org/printable/big-five-personality-test.pdf we have individual equations for calculating scores of each trait

bigfive_df['Extroversion Score'] = bigfive_df.apply(lambda ext: 20 + ext['EXT1'] - ext['EXT2'] + ext['EXT3'] - ext['EXT4'] + ext['EXT5'] - ext['EXT6'] + ext['EXT7'] - ext['EXT8'] + ext['EXT9'] - ext['EXT10'], axis=1)
bigfive_df['Agreeableness Score'] = bigfive_df.apply(lambda ext: 14 - ext['AGR1'] + ext['AGR2'] - ext['AGR3'] + ext['AGR4'] - ext['AGR5'] + ext['AGR6'] - ext['AGR7'] + ext['AGR8'] + ext['AGR9'] + ext['AGR10'], axis=1)
bigfive_df['Conscientiousness Score'] = bigfive_df.apply(lambda ext: 14 + ext['CSN1'] - ext['CSN2'] + ext['CSN3'] - ext['CSN4'] + ext['CSN5'] - ext['CSN6'] + ext['CSN7'] - ext['CSN8'] + ext['CSN9'] + ext['CSN10'], axis=1)
bigfive_df['Neuroticism Score'] = bigfive_df.apply(lambda ext: 38 - ext['EST1'] + ext['EST2'] - ext['EST3'] + ext['EST4'] - ext['EST5'] - ext['EST6'] - ext['EST7'] - ext['EST8'] - ext['EST9'] - ext['EST10'], axis=1)
bigfive_df['Openness Score'] = bigfive_df.apply(lambda ext: 8 + ext['OPN1'] - ext['OPN2'] + ext['OPN3'] - ext['OPN4'] + ext['OPN5'] - ext['OPN6'] + ext['OPN7'] + ext['OPN8'] + ext['OPN9'] + ext['OPN10'], axis=1)

CPU times: total: 2min 15s
Wall time: 2min 15s


In [27]:
# Negatively associated questions need to be flipped, done after the Score calculation
negative_answers = ['EXT2','EXT4','EXT6','EXT8','EXT10',
                   'AGR1','AGR3','AGR5','AGR7',
                   'CSN2','CSN4','CSN6','CSN8',
                   'EST1','EST3','EST5','EST6','EST7','EST8','EST9','EST10',
                   'OPN2','OPN4','OPN6'] 

bigfive_df[negative_answers] = bigfive_df[negative_answers].map(lambda x: 6 - x)

# Renaming the columns to include a negative "-" to annotate this is reversed
negative_answers_rename = {col: col + '-' for col in columns_to_flip}
bigfive_df = bigfive_df.rename(columns=negative_answers_rename)


bigfive_df.head()

,EXT1,EXT2-,EXT3,EXT4-,EXT5,EXT6-,EXT7,EXT8-,EXT9,EXT10-,EST1-,EST2,EST3-,EST4,EST5-,EST6-,EST7-,EST8-,EST9-,EST10-,AGR1-,AGR2,AGR3-,AGR4,AGR5-,AGR6,AGR7-,AGR8,AGR9,AGR10,CSN1,CSN2-,CSN3,CSN4-,CSN5,CSN6-,CSN7,CSN8-,CSN9,CSN10,OPN1,OPN2-,OPN3,OPN4-,OPN5,OPN6-,OPN7,OPN8,OPN9,OPN10,dateload,screenw,screenh,introelapse,testelapse,endelapse,country,Extroversion Score,Agreeableness Score,Conscientiousness Score,Neuroticism Score,Openness Score
0,4.00,5.00,5.00,4.00,5.00,5.00,5.00,4.00,4.00,5.00,5.00,4.00,2.00,2.00,4.00,4.00,4.00,4.00,3.00,4.00,4.00,5.00,4.00,4.00,4.00,3.00,4.00,4.00,3.00,4.00,3.00,2.00,3.00,4.00,2.00,2.00,4.00,4.00,4.00,4.00,5.00,5.00,4.00,5.00,4.00,5.00,5.00,3.00,4.00,5.00,2016-03-03 02:01:01,768.00,1024.00,9.00,234.00,6,GBR,36.00,29.00,22.00,26.00,35.00
1,3.00,1.00,3.00,2.00,3.00,3.00,2.00,1.00,1.00,1.00,4.00,3.00,2.00,1.00,3.00,5.00,4.00,5.00,3.00,5.00,5.00,4.00,5.00,5.00,5.00,5.00,3.00,4.00,5.00,3.00,3.00,4.00,5.00,3.00,3.00,5.00,3.00,3.00,5.00,3.00,1.00,4.00,4.00,4.00,3.00,5.00,4.00,2.00,5.00,3.00,2016-03-03 02:01:20,1360.00,768.00,12.00,179.00,11,MYS,10.00,34.00,27.00,25.00,25.00
2,2.00,3.00,4.00,2.00,3.00,4.00,1.00,3.00,2.00,1.00,2.00,4.00,2.00,2.00,4.00,4.00,4.00,4.00,5.00,3.00,5.00,4.00,5.00,4.00,4.00,4.00,5.00,4.00,4.00,3.00,4.00,4.00,2.00,4.00,3.00,3.00,4.00,4.00,4.00,2.00,5.00,5.00,2.00,5.00,4.00,4.00,5.00,3.00,4.00,4.00,2016-03-03 02:01:56,1366.00,768.00,3.00,186.00,7,GBR,15.00,32.00,24.00,24.00,31.00
3,2.00,4.00,2.00,3.00,4.00,4.00,2.00,2.00,1.00,2.00,3.00,3.00,3.00,2.00,3.00,4.00,4.00,4.00,2.00,3.00,4.00,4.00,3.00,4.00,4.00,4.00,4.00,4.00,3.00,4.00,2.00,2.00,4.00,2.00,1.00,4.00,2.00,3.00,1.00,4.00,4.00,4.00,5.00,4.00,3.00,5.00,4.00,4.00,3.00,3.00,2016-03-03 02:02:02,1920.00,1200.00,186.00,219.00,7,GBR,16.00,28.00,15.00,21.00,29.00
4,3.00,3.00,4.00,4.00,4.00,4.00,2.00,3.00,3.00,2.00,3.00,4.00,3.00,2.00,4.00,5.00,4.00,5.00,4.00,4.00,4.00,3.00,5.00,4.00,4.00,3.00,4.00,3.00,4.00,4.00,3.00,4.00,4.00,5.00,3.00,4.00,4.00,3.00,4.00,3.00,5.00,5.00,5.00,5.00,3.00,5.00,5.00,4.00,5.00,2.00,2016-03-03 02:03:12,1600.00,1000.00,4.00,196.00,3,SWE,22.00,28.00,27.00,28.00,34.00


In [28]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
bigfive_df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
EXT1,570277.00,2.58,1.24,1.00,1.00,3.00,4.00,5.00
EXT2-,570277.00,3.16,1.30,1.00,2.00,3.00,4.00,5.00
EXT3,570277.00,3.24,1.19,1.00,2.00,3.00,4.00,5.00
EXT4-,570277.00,2.78,1.21,1.00,2.00,3.00,4.00,5.00
EXT5,570277.00,3.25,1.25,1.00,2.00,3.00,4.00,5.00
EXT6-,570277.00,3.58,1.21,1.00,3.00,4.00,5.00,5.00
EXT7,570277.00,2.72,1.37,1.00,2.00,3.00,4.00,5.00
EXT8-,570277.00,2.53,1.24,1.00,1.00,2.00,3.00,5.00
EXT9,570277.00,2.95,1.32,1.00,2.00,3.00,4.00,5.00
EXT10-,570277.00,2.38,1.26,1.00,1.00,2.00,3.00,5.00


In [29]:
bigfive_df.nunique().T

EXT1                            5
EXT2-                           5
EXT3                            5
EXT4-                           5
EXT5                            5
EXT6-                           5
EXT7                            5
EXT8-                           5
EXT9                            5
EXT10-                          5
EST1-                           5
EST2                            5
EST3-                           5
EST4                            5
EST5-                           5
EST6-                           5
EST7-                           5
EST8-                           5
EST9-                           5
EST10-                          5
AGR1-                           5
AGR2                            5
AGR3-                           5
AGR4                            5
AGR5-                           5
AGR6                            5
AGR7-                           5
AGR8                            5
AGR9                            5
AGR10         

In [31]:
bigfive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570277 entries, 0 to 570276
Data columns (total 62 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   EXT1                     570277 non-null  float64
 1   EXT2-                    570277 non-null  float64
 2   EXT3                     570277 non-null  float64
 3   EXT4-                    570277 non-null  float64
 4   EXT5                     570277 non-null  float64
 5   EXT6-                    570277 non-null  float64
 6   EXT7                     570277 non-null  float64
 7   EXT8-                    570277 non-null  float64
 8   EXT9                     570277 non-null  float64
 9   EXT10-                   570277 non-null  float64
 10  EST1-                    570277 non-null  float64
 11  EST2                     570277 non-null  float64
 12  EST3-                    570277 non-null  float64
 13  EST4                     570277 non-null  float64
 14  EST5

In [30]:
bigfive_df.to_csv('bigfive_cleaned.csv', index=False) 